In [1]:
from copy import deepcopy
import time

In [2]:
def open_map(filename):
    file = open(filename)
    Map = file.read().split('\n')
    del Map[-1]
    # state = [
    #     [2,4], #Ambulance
    #     [[1, 8], [3, 2], [3, 9], [4,4]], #Patients
    #     {(1, 6): 1, (5, 2): 2, (5, 9): 1} #Hospitals
    # ] Sample State
    state = [[], [], {}]
    for i, row in enumerate(Map):
        for j, cell in enumerate(row):
            if cell == 'A':
                state[0] = [i, j]
            elif cell == 'P':
                state[1].append([i, j])
            elif cell.isdigit():
                state[2][i,j] = int(cell)
    start_state = state
    rawMap = deepcopy(Map)
    for i in range(len(rawMap)):
        rawMap[i] = rawMap[i].replace('1', ' ')
        rawMap[i] = rawMap[i].replace('2', ' ')
        rawMap[i] = rawMap[i].replace('3', ' ')
        rawMap[i] = rawMap[i].replace('P', ' ')
        rawMap[i] = rawMap[i].replace('A', ' ')
    return Map, rawMap, start_state

In [3]:
def printState(state):
    m = deepcopy(rawMap)
    a = state[0]
    m[a[0]] = m[a[0]][:a[1]] + 'A' + m[a[0]][a[1]+1:]
    for p in state[1]:
        m[p[0]] = m[p[0]][:p[1]] + 'P' + m[p[0]][p[1]+1:]
    for h in state[2]:
        m[h[0]] = m[h[0]][:h[1]] + str(state[2][h]) + m[h[0]][h[1]+1:]
    for row in m:
        print(row)
    print("---")


In [4]:
def print_steps(state, to_visit):
    s = state
    while s is not None:
        if print_steps:
            printState(s)
        s = to_visit[str(s)][0]


In [10]:
def IDS(start_state, depth):
    found = False
    goal_state = None
    found_depth = depth+1
    for d in range(16, depth):
        if found:
            break
        count = 0
        visited, queue, to_visit = set(), [start_state], dict()
        better_way = set()
        unique_state = 0
        total_state = 1
        to_visit[str(start_state)] = [None, 0]
        while queue:
            state = queue.pop(-1)
            unique_state += 1
            
            if str(state) not in visited or str(state) in better_way:
                if str(state) in better_way:
                    count += 1
                    better_way.remove(str(state))
                visited.add(str(state))
                i,j = state[0]
                if (len(state[1]) == 0) and found_depth > to_visit[str(state)][1] :
                    found = True
                    goal_state = state
                    found_depth = to_visit[str(state)][1]
                    
                if to_visit[str(state)][1] == d:
                    continue;
                for di, dj in [[1, 0], [0, -1], [-1, 0], [0, 1]]:
                    new_state = deepcopy(state)
        #             new_state.parent = state
                    ni = i+di; nj = j+dj
                    n2i = i+2*di; n2j = j+2*dj
                    if Map[ni][nj] != '#': # if neighbour is not wall
                        if [ni, nj] in state[1]: # if neighbour is a patient
                            if (n2i, n2j) in new_state[2] and new_state[2][n2i, n2j] > 0: #if 2neighbour is an hospital
                                new_state[0][0] += di
                                new_state[0][1] += dj
                                new_state[1].remove([ni, nj]) # remove patient
                                new_state[2][n2i, n2j] -= 1 # decrease hospital
                            elif ([n2i, n2j] not in new_state[1]) and  Map[n2i][n2j] != '#':
                                new_state[0][0] += di
                                new_state[0][1] += dj
                                new_state[1].remove([ni, nj]) # remove patient
                                new_state[1].append([n2i, n2j])
                                new_state[1].sort()
                            else:
                                continue
                        else:
                            new_state[0][0] += di
                            new_state[0][1] += dj
                        total_state += 1
                        if str(new_state) not in to_visit :
                            queue.append(new_state)
                            to_visit[str(new_state)] = [state, to_visit[str(state)][1] + 1]
                        elif to_visit[str(state)][1] + 1 < to_visit[str(new_state)][1]:
                            queue.append(new_state)
                            to_visit[str(new_state)] = [state, to_visit[str(state)][1] + 1]
                            better_way.add(str(new_state))

    return total_state, unique_state, to_visit, goal_state, found

In [6]:
Map, rawMap, start_state = open_map("test3.txt")
tic = time.time()
ts, us, to_visit, goal_state, found = IDS(start_state, 41)
print(ts, us, time.time() - tic, found)
print("Steps: ", to_visit[str(goal_state)][1])

243821 100013 63.80073547363281 True
Steps:  39


In [7]:
Map, rawMap, start_state = open_map("test2.txt")
tic = time.time()
ts, us, to_visit, goal_state, found = IDS(start_state, 30)
print(ts, us, time.time() - tic, found)
print("Steps: ", to_visit[str(goal_state)][1])

62719 27867 16.79711079597473 True
Steps:  27


In [11]:
Map, rawMap, start_state = open_map("test1.txt")
tic = time.time()
ts, us, to_visit, goal_state, found = IDS(start_state, 15)
print(ts, us, time.time() - tic, found)
print("Steps: ", to_visit[str(goal_state)][1])

UnboundLocalError: local variable 'total_state' referenced before assignment